In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


In [4]:
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [30]:
class Jetsons(Module):
    def __init__(self):
        self.writeHistFile=True
        self.counter = 0
    def beginJob(self,histFile=None,histDirName=None):
        Module.beginJob(self,histFile,histDirName)
        
        self.h_jets = ROOT.TH1D('h_jets', ';nJets;Events',   20, 0, 20)
        self.addObject(self.h_lumiblock)
        self.h_fatjets = ROOT.TH1D('h_fatjets', ';nFatJets;Events', 8, 0, 8)
        self.addObject(self.h_fatjets)
        self.h_subjets = ROOT.TH1D('h_subjets', ';nSubJets;Events', 16, 0, 16)
        self.addObject(self.h_subjets)
    #def endJob(self):
    #    pass #Cannot do pass in endJob if we write histograms... they don't get written! So do no override Module 
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        self.h_jets.Fill(event.nJet)
        self.h_fatjets.Fill(event.nFatJet)
        self.h_subjets.Fill(event.nSubJet)
        
        modulator = 1000 # how often to print an event when we do self.counter % modulator
        self.counter += 1
        if self.counter > 10:
            return False
        ###########################################
        ###### Basic Attributes of the Event ######
        ###########################################
        run = getattr(event, "run")
        lumi = getattr(event, "luminosityBlock")
        evt = getattr(event, "event")
        
        self.h_lumiblock.Fill(lumi)
        self.h_run.Fill(float(run))
        #if self.counter % modulator == 0:
        print("\n\nRun: {0:>8d} \tLuminosityBlock: {1:>8d} \tEvent: {2:>8d}".format(run,lumi,evt)) 
    
        ###########################################
        ###### Event Collections and Objects ######
        ###########################################
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        #taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        fatjets = Collection(event, "FatJet")
        subjets = Collection(event, "SubJet")
        met = Object(event, "MET")
        PV = Object(event, "PV")
        SV = Collection(event, "SV")
        
        ###############################
        ###### Time To Do Stuff? ######
        ###############################
        eventSum = ROOT.TLorentzVector()
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5.3f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2)) #getattr(PV, "chi2") also works
        print("==============================================================================")
        print("|| Secondary Vertices\tNumber: {0: >3d} \t\t\t\t\t    ||".format(len(SV)))
        if len(SV) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t\t    ||".format(
                "Pt", "Eta", "Phi", "Chi2", "nDoF", "Mass", "dLen"))
            for vert in SV:
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5.3f}\t\t    ||".format(
                    getattr(vert,"pt"),getattr(vert,"eta"),getattr(vert,"phi"),
                    getattr(vert,"chi2"),getattr(vert,"ndof"),getattr(vert,"mass"),getattr(vert,"dlen")))
        print("==============================================================================")
        print("|| Muons\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(muons)))
        if len(muons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso04"))
            for lep in muons:
                eventSum += lep.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                        getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso04_all")))
        print("==============================================================================")
        print("|| Electrons\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(electrons)))
        if len(electrons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso03"))
            for lep in electrons:
                eventSum += lep.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                        getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso03_all")))
        print("==============================================================================")
        print("|| Photons\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(photons)))
        if len(photons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}\t{4:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "mvaID", "PFRelIso03", "e ID", "JetID", "eVeto"))
            for gamma in photons:
                #eventSum += gamma.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5d}\t{5: >5d}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(gamma,"pt"),getattr(gamma,"eta"),getattr(gamma,"phi"),getattr(gamma,"mvaID_WP80"),
                        getattr(gamma,"electronIdx"),getattr(gamma,"jetIdx"),getattr(gamma,"electronVeto"),
                        getattr(gamma,"pfRelIso03_all")))
        print("==============================================================================")
        #for j in filter(self.jetSel,jets):
        print("|| AK4 Jets\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(jets)))
        print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}       ||".format(
                "Pt", "Eta", "Phi", "CSVv2", "CMVA", "JetID", "ChEmEF", "ChHEF"))
        for jet in jets:
            eventSum += jet.p4()
            print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5d}\t{6: >5.3f}\t{7: >5.3f}\t    ||".format(
                    getattr(jet,"pt"),getattr(jet,"eta"),getattr(jet,"phi"),getattr(jet,"btagCSVV2"),
                    getattr(jet,"btagCMVA"),getattr(jet,"jetId"),getattr(jet,"chEmEF"),getattr(jet,"chHEF")))
        print("==============================================================================")
        print("|| AK8 Jets\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(fatjets)))
        if len(fatjets) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}       ||".format(
                    "Pt", "Eta", "Phi", "CSVv2", "Mass", "MSDrp", "sJID1", "sJID2"))
            for fjet in fatjets:
                #eventSum += fjet.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5d}\t    ||".format(
                        getattr(fjet,"pt"),getattr(fjet,"eta"),getattr(fjet,"phi"),getattr(fjet,"btagCSVV2"),
                        getattr(fjet,"mass"),getattr(fjet,"msoftdrop"),getattr(fjet,"subJetIdx1"),
                        getattr(fjet,"subJetIdx2")))
        print("==============================================================================")
        print("|| AK8 SubJets\tNumber: {0: >3d}\t\t\t\t\t\t    ||".format(len(subjets)))
        if len(subjets) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t\t\t\t    ||".format(
                    "Pt", "Eta", "Phi", "CSVv2","Mass"))
            for sjet in subjets:
                #eventSum += fjet.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t\t\t\t    ||".format(
                        getattr(sjet,"pt"),getattr(sjet,"eta"),getattr(sjet,"phi"),getattr(sjet,"btagCSVV2"),
                        getattr(sjet,"mass")))
        print("==============================================================================")
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        
        ###########################################
        ###### Return True to pass the event ######
        ###########################################
        return True


In [34]:
#preselection="nMuon > 0 && nElectron > 0"
outputDir = "." #Just output where we run from for now
preselection="nSubJet > 3"
filePrefix = "root://cms-xrd-global.cern.ch/"
inputList = open("../data/Run2016/Run2016B_SM", "r")
files=[]
for line in inputList:
    files.append(filePrefix + str(line))
#for file in files:
#    print(file)
onefile = [files[0]] #Experiment with just one file for now...
#onefile = " root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root"
print(onefile)

[u'root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root\n']


In [35]:
p=PostProcessor(outputDir,files,cut=preselection,branchsel=None,modules=[Jetsons()],friend=False,postfix="_Jetsons", 
                jsonInput="../data/json/2016Golden.json",noOut=False,justcount=False,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName="histOut.root",
                histDirName="plots", outputbranchsel=None)
#"." is the outputdirectory, i.e. "Here" in this case
#jsonInput: json file in dictionary format {"RunNumberInt": [[lumilow,lumihigh],[lumi2low,lumi2high]], "RunNumber2Int":[[low,high]]}
#files: list ["fileone.root","filetwo.root"] of inputs, even if one file must be a list!
#branchsel: if non-None, selection of branches to not even activate/load into memory
#outputbranchsel: if non-None, selection of branches to still include in output (see noOut)
#noOut: If True, no output of skimmed data is written. If False, will write full data file fitting outputbranch selections and postfix concattenated to name
#postfix: string added to inputfile name to indicate this module processed it!
#histFileName: name of any output file for histograms created in your class, as above with runs and lumis
#histDirName: name INSIDE the "histFileName.root" file's directory structure!
#friend: Not tested
#provenance: not tested
#haddFileName: Not properly tested (tied together with fwkJobReport)
#fwkJobReport: only relevant with multiple files being added together? Not properly tested

In [36]:
p.run()

Will write selected trees to .
Pre-select 227 entries out of 11647 


Run:   273410 	LuminosityBlock:       50 	Event: 85573177
PV  X: 0.067 Y: 0.097 Z: -3.517 nDoF: 123.250 Chi^2: 0.754
|| Secondary Vertices	Number:   0 					    ||
|| Muons		Number:   0					    ||
|| Electrons	Number:   2						    ||
||	   Pt	  Eta	  Phi	 IP3d	  dXY	   dZ	JetID	PFRelIso03  ||
||	123.109	-0.425	0.985	0.004	-0.002	-0.004	    0	1.303	    ||
||	70.257	-0.405	0.995	0.004	0.001	0.005	    0	2.324	    ||
|| Photons	Number:   2						    ||
||	   Pt	  Eta	  Phi	mvaID	 e ID	JetID	eVeto	PFRelIso03  ||
||	124.727	-0.430	0.983	0.000	    0	    0	    0	0.262	    ||
||	70.584	-0.403	1.009	0.000	    1	    0	    1	2.311	    ||
|| AK4 Jets	Number:   2						    ||
||	   Pt	  Eta	  Phi	CSVv2	 CMVA	JetID	ChEmEF	ChHEF       ||
||	257.250	-0.436	0.999	0.097	-0.983	    3	0.289	0.104	    ||
||	249.375	-1.286	-2.062	0.174	-0.969	    3	0.000	0.305	    ||
|| AK8 Jets	Number:   2						    ||
||	   Pt	  Eta	  Phi	CSVv2	 Mass	MSDrp	s